*In this tutorial we will investigate two seperate companies and check if they are connected.*

There are instances where we may want to see if two companies are connected. We can do this by simply building a network for each company and comparing them to see if there are any common officers, addresses or companies.

Lets test this approach with two example companies, Zahawi & Zahawi Ltd (07285998) and Gorgeous Services Limited (05714521):

In [ ]:
import sugartrail
import pandas as pd

In [ ]:
sugartrail.api.basic_auth.username = ""

Create one network for Zahawi & Zahawi including some limits to reduce the number of possibly irrelevant connections:

In [ ]:
zahawi_connections = sugartrail.base.Network(company_id='07285998')
zahawi_connections.hop.officer_appointments_maxsize = 20
zahawi_connections.hop.officers_at_address_maxsize = 20
zahawi_connections.hop.companies_at_address_maxsize = 20

Create a second network for Gorgeous Services:

In [ ]:
gorgeous_connections = sugartrail.base.Network(company_id='05714521')
gorgeous_connections.hop.officer_appointments_maxsize = 20
gorgeous_connections.hop.officers_at_address_maxsize = 20
gorgeous_connections.hop.companies_at_address_maxsize = 20

We can now pass both networks to the `find_network_connections` method which returns any connections found between two networks. The method accepts two networks as input and an optional `max_depth` value (defaults to 5) which sets the maximum depth of network we will build for both. `find_network_connections` builds each network up to the `max_depth` value and completes when connections are found or the `max_depth` is reached.

In [ ]:
# zahawi_connections=sugartrail.base.Network(file=f'{sugartrail.const.networks_path}multinode/connection_check_a.json')

In [ ]:
# gorgeous_connections=sugartrail.base.Network(file=f'{sugartrail.const.networks_path}multinode/connection_check_b.json')

In [ ]:
connections = sugartrail.multinetwork.find_network_connections(zahawi_connections, gorgeous_connections)

Looks like a connection was found. We can see by the long string of characters that its an officer ID:

In [ ]:
connections

We can now trace the path from Zahawi & Zahawi to this connection:

In [ ]:
pd.DataFrame(zahawi_connections.find_path('g8BmvnpH8blqT87i93sgJeowx7I'))

... and the path from Gorgeous Connections to the connection:

In [ ]:
pd.DataFrame(gorgeous_connections.find_path('g8BmvnpH8blqT87i93sgJeowx7I'))

Reading both paths tells us how Zahawi & Zahawi connect to Gorgeous Connections. Zahawi & Zahawi has Nadhim Zahawi as an officer who has YOUGOV PLC as an appointment which has Benjamin Elliot as an officer who is also an officer of Gorgeous Services Limited.